## NOTEBOOK COPIA EJEMPLO NIXTLA hierarchicalforecast
Ejecución de notebook ejemplo base de nixtla - replicar código ejemplo y análisis extra para entender código y modelos ya hechos

LINK CÓDIGO ORIGINAL: https://nixtlaverse.nixtla.io/hierarchicalforecast/index.html

## Pararse en la raíz del repo

In [1]:
import os
from pathlib import Path

# Buscar el path al root del repo (buscando el archivo .git)
def find_repo_root(current_path: Path) -> Path:
    for parent in [current_path] + list(current_path.parents):
        if (parent / ".git").exists():
            return parent
    raise FileNotFoundError("No se encontró la raíz del repositorio (.git)")

# Cambiar el directorio de trabajo al root del repo
repo_root = find_repo_root(Path.cwd())
os.chdir(repo_root)
# print(f"Working directory set to repo root: {repo_root}")

## RUN CODE

In [2]:
# pip install hierarchicalforecast
# pip install datasetsforecast
# pip install statsforecast

In [3]:
import hierarchicalforecast

In [4]:
import pandas as pd
#obtain hierarchical dataset
from datasetsforecast.hierarchical import HierarchicalData

# compute base forecast no coherent
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive

#obtain hierarchical reconciliation methods and evaluation
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import evaluate
from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut, MinTrace
from utilsforecast.losses import mse

### 1. Load data

In [5]:
# Load TourismSmall dataset
Y_df, S, tags = HierarchicalData.load('./data', 'TourismSmall')
Y_df['ds'] = pd.to_datetime(Y_df['ds'])
S = S.reset_index(names="unique_id")

100%|█████████████████████████████████████| 1.30M/1.30M [00:00<00:00, 2.12MiB/s]
INFO:datasetsforecast.utils:Successfully downloaded datasets.zip, 1297274, bytes.
INFO:datasetsforecast.utils:Decompressing zip file...
INFO:datasetsforecast.utils:Successfully decompressed data/hierarchical/datasets.zip
100%|████████████████████████████████████████| 335k/335k [00:00<00:00, 709kiB/s]
INFO:datasetsforecast.utils:Successfully downloaded OldTraffic.zip, 335471, bytes.
INFO:datasetsforecast.utils:Decompressing zip file...
INFO:datasetsforecast.utils:Successfully decompressed data/hierarchical/OldTraffic.zip
100%|███████████████████████████████████████| 968k/968k [00:00<00:00, 1.64MiB/s]
INFO:datasetsforecast.utils:Successfully downloaded OldTourismLarge.zip, 967629, bytes.
INFO:datasetsforecast.utils:Decompressing zip file...
INFO:datasetsforecast.utils:Successfully decompressed data/hierarchical/OldTourismLarge.zip


In [6]:
# matriz S
S.shape

(89, 57)

In [7]:
S.head()

,unique_id,nsw-hol-city,nsw-hol-noncity,vic-hol-city,vic-hol-noncity,qld-hol-city,qld-hol-noncity,sa-hol-city,sa-hol-noncity,wa-hol-city,...,qld-oth-city,qld-oth-noncity,sa-oth-city,sa-oth-noncity,wa-oth-city,wa-oth-noncity,tas-oth-city,tas-oth-noncity,nt-oth-city,nt-oth-noncity
0,total,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,hol,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,vfr,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,bus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,oth,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [8]:
# matriz datos
Y_df.shape

(3204, 3)

In [9]:
Y_df.head()

,unique_id,ds,y
0,total,1998-03-31,84503
1,total,1998-06-30,65312
2,total,1998-09-30,72753
3,total,1998-12-31,70880
4,total,1999-03-31,86893


In [10]:
# tags - composición agregaciones de las series.
# esta configuración de agregaciones es la que debe reflejarse en la matriz S
tags

{'Country': array(['total'], dtype=object),
 'Country/Purpose': array(['hol', 'vfr', 'bus', 'oth'], dtype=object),
 'Country/Purpose/State': array(['nsw-hol', 'vic-hol', 'qld-hol', 'sa-hol', 'wa-hol', 'tas-hol',
        'nt-hol', 'nsw-vfr', 'vic-vfr', 'qld-vfr', 'sa-vfr', 'wa-vfr',
        'tas-vfr', 'nt-vfr', 'nsw-bus', 'vic-bus', 'qld-bus', 'sa-bus',
        'wa-bus', 'tas-bus', 'nt-bus', 'nsw-oth', 'vic-oth', 'qld-oth',
        'sa-oth', 'wa-oth', 'tas-oth', 'nt-oth'], dtype=object),
 'Country/Purpose/State/CityNonCity': array(['nsw-hol-city', 'nsw-hol-noncity', 'vic-hol-city',
        'vic-hol-noncity', 'qld-hol-city', 'qld-hol-noncity',
        'sa-hol-city', 'sa-hol-noncity', 'wa-hol-city', 'wa-hol-noncity',
        'tas-hol-city', 'tas-hol-noncity', 'nt-hol-city', 'nt-hol-noncity',
        'nsw-vfr-city', 'nsw-vfr-noncity', 'vic-vfr-city',
        'vic-vfr-noncity', 'qld-vfr-city', 'qld-vfr-noncity',
        'sa-vfr-city', 'sa-vfr-noncity', 'wa-vfr-city', 'wa-vfr-noncity',
     

In [11]:
tags.keys()

dict_keys(['Country', 'Country/Purpose', 'Country/Purpose/State', 'Country/Purpose/State/CityNonCity'])

In [12]:
tags['Country']

array(['total'], dtype=object)

In [13]:
tags['Country/Purpose']

array(['hol', 'vfr', 'bus', 'oth'], dtype=object)

In [14]:
tags['Country/Purpose/State']

array(['nsw-hol', 'vic-hol', 'qld-hol', 'sa-hol', 'wa-hol', 'tas-hol',
       'nt-hol', 'nsw-vfr', 'vic-vfr', 'qld-vfr', 'sa-vfr', 'wa-vfr',
       'tas-vfr', 'nt-vfr', 'nsw-bus', 'vic-bus', 'qld-bus', 'sa-bus',
       'wa-bus', 'tas-bus', 'nt-bus', 'nsw-oth', 'vic-oth', 'qld-oth',
       'sa-oth', 'wa-oth', 'tas-oth', 'nt-oth'], dtype=object)

In [15]:
tags['Country/Purpose/State/CityNonCity']

array(['nsw-hol-city', 'nsw-hol-noncity', 'vic-hol-city',
       'vic-hol-noncity', 'qld-hol-city', 'qld-hol-noncity',
       'sa-hol-city', 'sa-hol-noncity', 'wa-hol-city', 'wa-hol-noncity',
       'tas-hol-city', 'tas-hol-noncity', 'nt-hol-city', 'nt-hol-noncity',
       'nsw-vfr-city', 'nsw-vfr-noncity', 'vic-vfr-city',
       'vic-vfr-noncity', 'qld-vfr-city', 'qld-vfr-noncity',
       'sa-vfr-city', 'sa-vfr-noncity', 'wa-vfr-city', 'wa-vfr-noncity',
       'tas-vfr-city', 'tas-vfr-noncity', 'nt-vfr-city', 'nt-vfr-noncity',
       'nsw-bus-city', 'nsw-bus-noncity', 'vic-bus-city',
       'vic-bus-noncity', 'qld-bus-city', 'qld-bus-noncity',
       'sa-bus-city', 'sa-bus-noncity', 'wa-bus-city', 'wa-bus-noncity',
       'tas-bus-city', 'tas-bus-noncity', 'nt-bus-city', 'nt-bus-noncity',
       'nsw-oth-city', 'nsw-oth-noncity', 'vic-oth-city',
       'vic-oth-noncity', 'qld-oth-city', 'qld-oth-noncity',
       'sa-oth-city', 'sa-oth-noncity', 'wa-oth-city', 'wa-oth-noncity',
       

### 2. Entender matriz S

In [16]:
# se ve que hay 87 filas y 57 columnas (cada fila y cada columna valor único), 
# entonces, la intuiación es que en las filas primero aparecen las agregaciones y luego aparecen las series individuales que conforman las agregaciones

#
# filas que representan agregaciones de series: tienen más de un valor 1 (en las columnas)
# filas que representan series individuales: tienen un único valor 1 (en las columnas) ya que corresponde a una serie individual

In [17]:
# obtener listado de series en columnas y filas
list_columns = S.set_index('unique_id').columns.tolist()
list_rows = S['unique_id'].values.tolist()
print('list_columns: ', len(list_columns))
print('list_rows: ', len(list_rows))

list_columns:  56
list_rows:  89


In [18]:
# valores que están en las columnas que no están en las filas. deberían estar todos porque en las columnas estan las series individuales
list_columns_minus_list_rows = list(set(list_columns) - set(list_rows))
len(list_columns_minus_list_rows)

0

In [19]:
# valores las filas que no están en las columnas. se deberían retornar las series agregadas, ya que las series agregadas están en las filas
list_rows_minus_list_columns = list(set(list_rows) - set(list_columns))
len(list_rows_minus_list_columns)

33

In [20]:
# validar: al sumar las series individuales en las columnas más las series de las filas que representan la agregación de series debe ser igual al total de filas
len(list_rows_minus_list_columns) + len(list_columns)

89

In [21]:
# mostrar el valor de ejemplo de una serie agregación de series individuales
show_serie_example = S[S['unique_id'] == 'wa-oth'].reset_index(drop = True).iloc[0]
# show_serie_example

In [22]:
# mostrar el valor de ejemplo de una serie individual
show_serie_example = S[S['unique_id'] == 'nt-oth-noncity'].reset_index(drop = True).iloc[0]
# show_serie_example

### 3. Entender datos series

In [23]:
Y_df.head(3)

,unique_id,ds,y
0,total,1998-03-31,84503
1,total,1998-06-30,65312
2,total,1998-09-30,72753


In [24]:
# se ve que hay 89 series en los datos, lo que corresponde al total de series que existen en la matriz S
cantidad_series = Y_df['unique_id'].unique().shape
cantidad_series

(89,)

In [25]:
# cantidad de datos en cada serie
cantidad_datos_cada_serie = Y_df[Y_df['unique_id'] == 'total']['ds'].unique().shape
cantidad_datos_cada_serie

(36,)

In [26]:
# validar que todas las series tienen la misma cantidad de datos
# cantidad de datos (de una serie) * cantidad de series = tamaño dataframe
print( cantidad_datos_cada_serie[0] * cantidad_series[0] )
print( Y_df.shape[0] )
print( cantidad_datos_cada_serie[0] * cantidad_series[0] == Y_df.shape[0] )

3204
3204
True


### 4. split train/test datos series
Para test las últimas 4 observaciones

In [27]:
# print cantidad de observaciones en cada serie
cantidad_datos_cada_serie[0]

36

In [28]:
#split train/test sets
Y_test_df  = Y_df.groupby('unique_id').tail(4)
Y_train_df = Y_df.drop(Y_test_df.index)

print('shape train: ', Y_train_df.shape)
print('shape test: ', Y_test_df.shape)

shape train:  (2848, 3)
shape test:  (356, 3)


In [29]:
89*4 # 89 series, 4 observaciones por serie -> 356 datos de tests

356

### 5. Compute base auto-ARIMA predictions

In [30]:
# definir modelo
fcst = StatsForecast(models = [AutoARIMA(season_length=4), Naive()],
                     freq = 'QE', 
                     n_jobs = -1)
fcst

StatsForecast(models=[AutoARIMA,Naive])

In [31]:
# entrenar y predecir 4 observaciones futuras
Y_hat_df = fcst.forecast(df = Y_train_df, h = 4)

In [32]:
fcst

StatsForecast(models=[AutoARIMA,Naive])

In [33]:
Y_hat_df.shape

(356, 4)

In [34]:
Y_hat_df.head()

,unique_id,ds,AutoARIMA,Naive
0,bus,2006-03-31,8918.476562,11547.0
1,bus,2006-06-30,9581.923828,11547.0
2,bus,2006-09-30,11194.674805,11547.0
3,bus,2006-12-31,10678.956055,11547.0
4,hol,2006-03-31,42805.335938,26418.0


### 6. Calcular métricas de las series

In [35]:
from sklearn.metrics import mean_absolute_error

In [36]:
# definir lista con el orden usado en las series
list_series = Y_test_df['unique_id'].unique().tolist()

# ordenar Y_PRED por unique_id. ordenar las series transformando a tipo categorical y luego regresar a formato string
Y_hat_df['unique_id'] = pd.Categorical(Y_hat_df['unique_id'], categories=list_series, ordered=True)
Y_hat_df = Y_hat_df.sort_values(['unique_id', 'ds'], ascending = [True, True]).reset_index(drop=True) # ordenar por orden de las series y en fecha ascendente
Y_hat_df['unique_id'] = Y_hat_df['unique_id'].astype(str)

In [37]:
# calcular MAE considerando todos los volumenes de cada serie iguales
mae_arima = mean_absolute_error(y_true = Y_test_df['y'], 
                    y_pred = Y_hat_df['AutoARIMA'])
mae_arima

312.42621037933264

In [38]:
mae_naive = mean_absolute_error(y_true = Y_test_df['y'], 
                    y_pred = Y_hat_df['Naive'])
mae_naive

629.7078651685393

### 7. Reconciliar forecast

In [39]:
# Reconcile the base predictions
reconcilers = [
    BottomUp(),
    TopDown(method='forecast_proportions'),
    MiddleOut(middle_level='Country/Purpose/State',
              top_down_method='forecast_proportions'),
    MinTrace(method = 'ols', nonnegative = True)
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_train_df,
                          S=S, tags=tags)

### 8. Analizar fcst conciliados

#### 8.1 Definir variables. Validar resultados serie agregada y series individuales que la componen

In [40]:
# generar lista con el id de todas las series - individuales y agregadas
list_all_series = S['unique_id'].unique().tolist()
len(list_all_series)

89

In [41]:
# defir variable nombre del modelo obtenido con nixtla. Naive ES EL MÁS SIMPLE DE ANALIZAR PARA VER 
name_model = 'AutoARIMA'

#### 8.2 Validar bottomUp

In [42]:
## SERIES A ANALIZAR - TOMO DE EJEMPLO
# SERIE AGREGADA: 'nsw-hol'
# SERIES INDIVIDUALES QUE LA COMPONEN de menor nivel: 'nsw-hol-city', 'nsw-hol-noncity'


# definir serie agregada a analizar
id_serie_agregada = 'nsw-hol'
list_id_series_individuales = ['nsw-hol-city', 'nsw-hol-noncity']

# filtrar forecast serie agregada
Y_rec_df_serie_agregada = Y_rec_df[Y_rec_df['unique_id'] == id_serie_agregada]

# filtrar forecast series individuales
Y_rec_df_series_individuales = Y_rec_df[Y_rec_df['unique_id'].isin(list_id_series_individuales)]

In [43]:
### validar que el forecast bottom/up es el mismo que el forecast original por las series univariadas
Y_rec_df_series_individuales[f'{name_model}/BottomUp'] == Y_rec_df_series_individuales[f'{name_model}']

132    True
133    True
134    True
135    True
136    True
137    True
138    True
139    True
dtype: bool

In [44]:
### validar que la suma de los forecast individuales bottom/up es corresponde al forecast bottom/up de la serie agregada
suma_forecast_series_individuales = Y_rec_df_series_individuales[Y_rec_df_series_individuales['unique_id'] == list_id_series_individuales[0]][f'{name_model}'].values + \
                                    Y_rec_df_series_individuales[Y_rec_df_series_individuales['unique_id'] == list_id_series_individuales[1]][f'{name_model}'].values
Y_rec_df_serie_agregada[f'{name_model}/BottomUp'].values == suma_forecast_series_individuales

# obs se ve que diferencia es por calculo de decimales. [True, True,  True, True]

array([False, False,  True, False])

In [45]:
Y_rec_df_serie_agregada[f'{name_model}/BottomUp'].values

array([13671.03051758,  8038.72875977,  7902.36767578,  7828.56530762])

In [46]:
suma_forecast_series_individuales

array([13671.03  ,  8038.7285,  7902.3677,  7828.5654], dtype=float32)

### 8.3 Validar TopDown

In [47]:
## SERIES A ANALIZAR - TOMO DE EJEMPLO
# SERIE AGREGADA la más agregada - total: 'total'
# SERIES INDIVIDUALES QUE LA COMPONEN de menor nivel: 'hol', 'vfr', 'bus', 'oth'


# definir serie agregada a analizar
id_serie_agregada = 'total'
list_id_series_individuales = ['hol', 'vfr', 'bus', 'oth']

# filtrar forecast serie agregada
Y_rec_df_serie_agregada = Y_rec_df[Y_rec_df['unique_id'] == id_serie_agregada]

# filtrar forecast series individuales
Y_rec_df_series_individuales = Y_rec_df[Y_rec_df['unique_id'].isin(list_id_series_individuales)]

In [48]:
### validar que el forecast agregado top/down sea el mismo que el forecast de la serie individual
Y_rec_df_serie_agregada[f'{name_model}/TopDown_method-forecast_proportions'] == Y_rec_df_serie_agregada[f'{name_model}']

0    True
1    True
2    True
3    True
dtype: bool

In [49]:
### validar que las series individuales se calcula un share y con eso se apertura.
# falta validar si el share es por horizonte o se mantiene para todos los horizontes
# con el modelo arima no me cuadró - REVISAR POR QUÉ NO CUADRA CON EL MODELO ARIMA

In [50]:
26418.0 + 19954.0 + 11547.0 + 5473.0 # OBTENER total de forecast series individuales

63392.0

In [51]:
26418.0 / 63392.0 # calcular share primera serie individuale dado el total forecastado por todas las series individuales

0.4167402826855124

In [52]:
0.4167402826855124 * 63392.0 # multiplicar ratio calculado por fcst inviduales por forecast agregado

26418.0

#### 8.4 Validar MiddleOut

In [53]:
# MiddleOut

#### 8.5 Validar MinTrace

In [54]:
## SERIES A ANALIZAR - TOMO DE EJEMPLO
# SERIE AGREGADA: 'nsw-hol'
# SERIES INDIVIDUALES QUE LA COMPONEN de menor nivel: 'nsw-hol-city', 'nsw-hol-noncity'


# definir serie agregada a analizar
id_serie_agregada = 'nsw-hol'
list_id_series_individuales = ['nsw-hol-city', 'nsw-hol-noncity']

# filtrar forecast serie agregada
Y_rec_df_serie_agregada = Y_rec_df[Y_rec_df['unique_id'] == id_serie_agregada]

# filtrar forecast series individuales
Y_rec_df_series_individuales = Y_rec_df[Y_rec_df['unique_id'].isin(list_id_series_individuales)]

In [55]:
# validar que sumar el forecast mintrace de las series individuales se obtiene el forecast mintrace de las series agregadas
# OJO AHORA AL SER NO SER TOP/DOWN Y BOTTON-UP EL FORECAST ORIGINAL NO SE MANTIENE PARA NINGUNO DE LOS 2

In [56]:

fcst_mintrace_agregada = Y_rec_df_serie_agregada[f'{name_model}/MinTrace_method-ols_nonnegative-True'].reset_index(drop = True)

fcst_mintrace_serie_individuales = Y_rec_df_series_individuales[Y_rec_df_series_individuales['unique_id'] == list_id_series_individuales[0]][f'{name_model}/MinTrace_method-ols_nonnegative-True'].reset_index(drop = True) + \
                                    Y_rec_df_series_individuales[Y_rec_df_series_individuales['unique_id'] == list_id_series_individuales[1]][f'{name_model}/MinTrace_method-ols_nonnegative-True'].reset_index(drop = True)

fcst_mintrace_agregada == fcst_mintrace_serie_individuales

0    True
1    True
2    True
3    True
Name: AutoARIMA/MinTrace_method-ols_nonnegative-True, dtype: bool

In [57]:
fcst_mintrace_agregada

0    13803.899292
1     8147.614502
2     7989.051270
3     7837.082153
Name: AutoARIMA/MinTrace_method-ols_nonnegative-True, dtype: float64

In [58]:
fcst_mintrace_serie_individuales

0    13803.899292
1     8147.614502
2     7989.051270
3     7837.082153
Name: AutoARIMA/MinTrace_method-ols_nonnegative-True, dtype: float64

In [59]:
# se puede observar que sí se conserva el volumen

### 8. Calcular métricas luego de reconciliar forecast

In [60]:
# BASELINE
# calcular MAE considerando todos los volumenes de cada serie iguales
mae_arima = mean_absolute_error(y_true = Y_test_df['y'], 
                    y_pred = Y_hat_df['AutoARIMA'])
mae_arima

312.42621037933264

In [61]:
# BASELINE
# calcular MAE considerando todos los volumenes de cada serie iguales
mean_absolute_error(y_true = Y_test_df['y'], 
                    y_pred = Y_rec_df['AutoARIMA']) # CAMBIAR DF POR EL DF CON LOS FORECAST RECONCILIADOS

312.42621037933264

In [62]:
# calcular MAE: AutoARIMA/BottomUp
mean_absolute_error(y_true = Y_test_df['y'], 
                    y_pred = Y_rec_df['AutoARIMA/BottomUp'])

266.671261835634

In [63]:
# calcular MAE: AutoARIMA/TopDown_method-forecast_proportions
mean_absolute_error(y_true = Y_test_df['y'], 
                    y_pred = Y_rec_df['AutoARIMA/TopDown_method-forecast_proportions'])

319.8625555868647

In [64]:
# calcular MAE: AutoARIMA/MiddleOut_middle_level-Country/Purpose/State_top_down_method-forecast_proportions
mean_absolute_error(y_true = Y_test_df['y'], 
                    y_pred = Y_rec_df['AutoARIMA/MiddleOut_middle_level-Country/Purpose/State_top_down_method-forecast_proportions'])

301.61157307052514

In [65]:
# calcular MAE: AutoARIMA/MinTrace_method-ols_nonnegative-True
mean_absolute_error(y_true = Y_test_df['y'], 
                    y_pred = Y_rec_df['AutoARIMA/MinTrace_method-ols_nonnegative-True'])

332.9575600249044